# **Наполнение графа знаний значениями:**

Подключение (установка) нужных библиотек:

In [109]:
from re import T
import subprocess
import sys
import ssl
import io
import os
try:
    import steamspypi
except:
    subprocess.check_call([sys.executable, "-m", "pip", "install", "steamspypi"])
finally:
    import steamspypi
try:
    import requests
except:
    subprocess.check_call([sys.executable, "-m", "pip", "install", "requests"])
finally:
    import requests
try:
    from howlongtobeatpy import HowLongToBeat as hltb
except:
    subprocess.check_call([sys.executable, "-m", "pip", "install", "howlongtobeatpy"])
finally:
    from howlongtobeatpy import HowLongToBeat as hltb
try:
    import rdflib
    from rdflib import Graph, Literal, RDF, URIRef
    from rdflib.namespace import FOAF , XSD
except:
    subprocess.check_call([sys.executable, "-m", "pip", "install", "rdflib"])
finally:
    import rdflib
    from rdflib import Graph, Literal, RDF, URIRef, Namespace
    from rdflib.namespace import FOAF , XSD


Инициализация графа:

In [110]:
gr = Graph()
gr.parse("SteamKB.owl", format="xml")

<Graph identifier=N4137f5a9c65d4c2e867ef49dd1a17b06 (<class 'rdflib.graph.Graph'>)>

Для удобства добавления значения создаём класс "Игра":

In [111]:
class Game:
    appid = 0
    tag = []
    name = ""
    peak_activity = ""
    avg_players = 0
    owners = 0
    rating = 0.0
    price = 0
    genre = []
    playtime = 0

Функция парсинга SteamDB (нужна для определения времени суток, в которых активность пиковая):

In [112]:
def parseSteamDBTime(srch):
    cut = srch.split("<h2>Peak daily concurrent players:</h2>")[1].split("<svg>")[1]
    
    night = int(cut.split("[")[3].split(", ")[1])
    morn = int(cut.split("[")[4].split(", ")[1])
    day = int(cut.split("[")[5].split(", ")[1])
    even = int(cut.split("[")[6].split(", ")[1])
    m = max(night, morn, day, even)
    s = ""
    if m == night:
        s = ("Night")
    if m == morn:
        s = ("Morning")
    if m == day:
        s = ("Day")
    if m == even:
        s = ("Evening")
    return s

Функция поиска всех необходимых данных. Принимает AppID приложения, возвращает объект класса "Игра":

In [113]:
def search(i):
    g = Game
    data_request = dict()
    data_request['request'] = 'appdetails'
    data_request['appid'] = str(i)
    try:
        try:
            d = steamspypi.download(data_request)
            result = hltb().search(d['name'])
            try:
                if d['name'] != None and d['median_forever'] > 0 and result != None:
                    steamspy = requests.get('https://steamspy.com/app/'+str(d['appid']))
                    g.appid = i
                    g.name = d['name']
                    try:
                        g.peak_activity = parseSteamDBTime(steamspy.text)
                    except:
                        g.peak_activity = "Evening"
                    g.avg_players = int(d['median_forever'])
                    g.owners = int(d['owners'].replace(",", "").split(" ")[0])
                    if float(d['negative']) > 0.0:
                        g.rating = float(d['positive']) / float(d['negative'])
                    else:
                        g.rating = 1000000.0
                    g.price = int(d['price'])
                    g.genre = d['genre'].split(", ")
                    g.tag = d['tags']
                    try:
                        g.playtime = int(str(result[0].gameplay_main).replace('½', ''))
                        if g.playtime == -1:
                            g.playtime = 9999
                    except:
                        g.playtime = 9999
                        print("HLTB parsing error!")
                    return g
            except KeyError:
                print(str(i)+":\t"+'KeyError!')

        except ssl.SSLError:
            print(str(i)+":\t"+'SSLError!')
    except requests.exceptions.ConnectionError:
        print(str(i)+":\t"+'ConnectionError!')
    return g


Функция добавления в граф новой игры. Принимает объект класса "Игра":



In [114]:
def rdfAdd(gm):
    ns = ('http://www.semanticweb.org/svg/ontologies/2021/4/untitled-ontology-3#')
    g = gr
    ref = URIRef(Namespace(ns+str(gm.appid)))
    g.add((ref, RDF.type, URIRef(ns+"Game")))
    for t in gm.tag:
        g.add((URIRef(ns+t), RDF.type, URIRef(ns+"Tag")))
        g.add((ref, URIRef(ns+"hasTag"), URIRef(ns+t)))
    for t in gm.genre:
        g.add((URIRef(ns+t), RDF.type, URIRef(ns+"Genre")))
        g.add((ref, URIRef(ns+"hasGenre"), URIRef(ns+t)))
    
    #g.add((URIRef(ns+gm.name), RDF.type, URIRef(ns+"name")))
    #g.add(ref, URIRef(ns+"name"),Literal(gm.name, datatype=XSD.string))

    g.add((ref, URIRef(ns+"appid"), Literal(gm.appid, datatype=XSD.int)))
    g.add((ref, URIRef(ns+"name"), Literal(gm.name, datatype=XSD.string)))
    g.add((ref, URIRef(ns+"owners"), Literal(gm.owners, datatype=XSD.int)))
    g.add((ref, URIRef(ns+"peak_activity"), Literal(gm.peak_activity, datatype=XSD.string)))
    g.add((ref, URIRef(ns+"players_amt"), Literal(gm.avg_players, datatype=XSD.int)))
    g.add((ref, URIRef(ns+"playtime"), Literal(gm.playtime, datatype=XSD.int)))
    g.add((ref, URIRef(ns+"price"), Literal(gm.price, datatype=XSD.int)))
    g.add((ref, URIRef(ns+"rating"), Literal(gm.rating, datatype=XSD.float)))
    g.serialize(destination='output2.owl', format='xml')


Цикл ввода данных в граф:

In [115]:
opType = 'load'
start = 0
f = open('dataset2.csv', 'wb' )
s = ""
s+='\n\r'
f.write(s[:-1].encode())
fn = open('2000Games.csv', 'r')
lns = fn.readlines()
for i in range(1,10):
    try:
        id=int(lns[i].split(",")[0])
        g = search(id)
        rdfAdd(g)
        print(str(i+1)+": "+g.name)
    except:
        print("Error!")
gr.serialize(destination='output2.owl', format='xml')

http://www.semanticweb.org/svg/ontologies/2021/4/untitled-ontology-3#Battle Royale does not look like a valid URI, trying to serialize this will break.
http://www.semanticweb.org/svg/ontologies/2021/4/untitled-ontology-3#Battle Royale does not look like a valid URI, trying to serialize this will break.
http://www.semanticweb.org/svg/ontologies/2021/4/untitled-ontology-3#Third-Person Shooter does not look like a valid URI, trying to serialize this will break.
http://www.semanticweb.org/svg/ontologies/2021/4/untitled-ontology-3#Third-Person Shooter does not look like a valid URI, trying to serialize this will break.
http://www.semanticweb.org/svg/ontologies/2021/4/untitled-ontology-3#Online Co-Op does not look like a valid URI, trying to serialize this will break.
http://www.semanticweb.org/svg/ontologies/2021/4/untitled-ontology-3#Online Co-Op does not look like a valid URI, trying to serialize this will break.
http://www.semanticweb.org/svg/ontologies/2021/4/untitled-ontology-3#Early A

HLTB parsing error!
2: PLAYERUNKNOWN'S BATTLEGROUNDS


http://www.semanticweb.org/svg/ontologies/2021/4/untitled-ontology-3#Free to Play does not look like a valid URI, trying to serialize this will break.
http://www.semanticweb.org/svg/ontologies/2021/4/untitled-ontology-3#Free to Play does not look like a valid URI, trying to serialize this will break.
http://www.semanticweb.org/svg/ontologies/2021/4/untitled-ontology-3#Online Co-Op does not look like a valid URI, trying to serialize this will break.
http://www.semanticweb.org/svg/ontologies/2021/4/untitled-ontology-3#Online Co-Op does not look like a valid URI, trying to serialize this will break.
http://www.semanticweb.org/svg/ontologies/2021/4/untitled-ontology-3#Tower Defense does not look like a valid URI, trying to serialize this will break.
http://www.semanticweb.org/svg/ontologies/2021/4/untitled-ontology-3#Tower Defense does not look like a valid URI, trying to serialize this will break.
http://www.semanticweb.org/svg/ontologies/2021/4/untitled-ontology-3#Character Customization

3: Dota 2


http://www.semanticweb.org/svg/ontologies/2021/4/untitled-ontology-3#Online Co-Op does not look like a valid URI, trying to serialize this will break.
http://www.semanticweb.org/svg/ontologies/2021/4/untitled-ontology-3#Online Co-Op does not look like a valid URI, trying to serialize this will break.
http://www.semanticweb.org/svg/ontologies/2021/4/untitled-ontology-3#Free to Play does not look like a valid URI, trying to serialize this will break.
http://www.semanticweb.org/svg/ontologies/2021/4/untitled-ontology-3#Free to Play does not look like a valid URI, trying to serialize this will break.
http://www.semanticweb.org/svg/ontologies/2021/4/untitled-ontology-3#Online Co-Op does not look like a valid URI, trying to serialize this will break.
http://www.semanticweb.org/svg/ontologies/2021/4/untitled-ontology-3#Online Co-Op does not look like a valid URI, trying to serialize this will break.
http://www.semanticweb.org/svg/ontologies/2021/4/untitled-ontology-3#Free to Play does not loo

...

8: Path of Exile


http://www.semanticweb.org/svg/ontologies/2021/4/untitled-ontology-3#Free to Play does not look like a valid URI, trying to serialize this will break.
http://www.semanticweb.org/svg/ontologies/2021/4/untitled-ontology-3#Free to Play does not look like a valid URI, trying to serialize this will break.
http://www.semanticweb.org/svg/ontologies/2021/4/untitled-ontology-3#Looter Shooter does not look like a valid URI, trying to serialize this will break.
http://www.semanticweb.org/svg/ontologies/2021/4/untitled-ontology-3#Looter Shooter does not look like a valid URI, trying to serialize this will break.
http://www.semanticweb.org/svg/ontologies/2021/4/untitled-ontology-3#Third-Person Shooter does not look like a valid URI, trying to serialize this will break.
http://www.semanticweb.org/svg/ontologies/2021/4/untitled-ontology-3#Third-Person Shooter does not look like a valid URI, trying to serialize this will break.
http://www.semanticweb.org/svg/ontologies/2021/4/untitled-ontology-3#Onlin

9: Warframe


http://www.semanticweb.org/svg/ontologies/2021/4/untitled-ontology-3#Free to Play does not look like a valid URI, trying to serialize this will break.
http://www.semanticweb.org/svg/ontologies/2021/4/untitled-ontology-3#Free to Play does not look like a valid URI, trying to serialize this will break.
http://www.semanticweb.org/svg/ontologies/2021/4/untitled-ontology-3#Hero Shooter does not look like a valid URI, trying to serialize this will break.
http://www.semanticweb.org/svg/ontologies/2021/4/untitled-ontology-3#Hero Shooter does not look like a valid URI, trying to serialize this will break.
http://www.semanticweb.org/svg/ontologies/2021/4/untitled-ontology-3#Online Co-Op does not look like a valid URI, trying to serialize this will break.
http://www.semanticweb.org/svg/ontologies/2021/4/untitled-ontology-3#Online Co-Op does not look like a valid URI, trying to serialize this will break.
http://www.semanticweb.org/svg/ontologies/2021/4/untitled-ontology-3#Free to Play does not loo

10: Team Fortress 2


# **SPARQL-запросы:**

Самые недооценённые игры:

In [ ]:
result = gr.query(
    """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX kg: <http://www.semanticweb.org/svg/ontologies/2021/4/untitled-ontology-3#>
SELECT $id $name $players_amt $rating WHERE
{
	?id kg:name ?name.
	?id kg:players_amt ?players_amt.
	?id kg:rating ?rating
	FILTER (?players_amt >= 5000 && ?rating <2)
}ORDER BY DESC(?players_amt) LIMIT 10""")

for row in result:
    print(row['name']+":\t"+row["rating"])

PLAYERUNKNOWN'S BATTLEGROUNDS:	1.1837943057985774


Самые переоценённые игры:

In [ ]:
result = gr.query(
    """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX kg: <http://www.semanticweb.org/svg/ontologies/2021/4/untitled-ontology-3#>
SELECT $id $name $players_amt $rating WHERE
{
	?id kg:name ?name.
	?id kg:players_amt ?players_amt.
	?id kg:rating ?rating
	FILTER (?players_amt < 5000 && ?rating >= 2)
}ORDER BY ASC(?players_amt) LIMIT 10""")
for row in result:
    print(row['name']+":\t"+row["rating"])

Team Fortress 2:	15.44155624947253
Warframe:	9.77119242011894
Path of Exile:	12.753925812749342
Dota 2:	5.320749468999696
Tom Clancy's Rainbow Six Siege:	7.386901596963752


Во что стоит играть утром:

In [ ]:
result = gr.query(
    """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX kg: <http://www.semanticweb.org/svg/ontologies/2021/4/untitled-ontology-3#>
SELECT $id $name WHERE
{
	?genre rdf:type kg:Genre.
	?id rdf:type kg:Game.

	?id kg:name ?name.
	?id kg:peak_activity ?peak_activity.
	FILTER regex(str(?peak_activity), "Morning")
	FILTER regex(str(?genre), "Massively%20Multiplayer")
}LIMIT 10
""")
for row in result:
    print(row['name'])

Топ-10 наиболее выгодных игр в жанре "Стратегия":

In [ ]:
result = gr.query(
    """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX kg: <http://www.semanticweb.org/svg/ontologies/2021/4/untitled-ontology-3#>
SELECT $id $name $price $rating WHERE
{
	?genre rdf:type kg:Genre.
	?id rdf:type kg:Game.
	?id kg:price ?price.
	?id kg:rating ?rating.
	?id kg:name ?name
	FILTER regex(str(?genre), "Strategy")
	FILTER (?price > 0)
}ORDER BY DESC(?rating / ?price)
LIMIT 10
""")
for row in result:
    print(row['name']+", "+row['price']+" RUR. Rating: "+row['rating'])

Tom Clancy's Rainbow Six Siege, 1999 RUR. Rating: 7.386901596963752
Grand Theft Auto V, 2998 RUR. Rating: 4.717808818246659
PLAYERUNKNOWN'S BATTLEGROUNDS, 2999 RUR. Rating: 1.1837943057985774


# **Reasoner**

In [116]:
try:
    from owlready2 import *
except:
    subprocess.check_call([sys.executable, "-m", "pip", "install", "owlready2"])
finally:
    from owlready2 import *

onto = get_ontology("output2.owl").load()
with onto:
  sync_reasoner()
  print(str(onto.different_individuals()))

* Owlready2 * Running HermiT...
    java -Xmx2000M -cp /usr/local/lib/python3.7/dist-packages/owlready2/hermit:/usr/local/lib/python3.7/dist-packages/owlready2/hermit/HermiT.jar org.semanticweb.HermiT.cli.CommandLine -c -O -D -I file:////tmp/tmps2lliq2_


<generator object _GraphManager.different_individuals at 0x7f8413316050>


* Owlready2 * HermiT took 0.8975827693939209 seconds
* Owlready * (NB: only changes on entities loaded in Python are shown, other changes are done but not listed)


# **SHACL**

Генерация шейпов:

In [ ]:
try:
    import shaclgen
except:
    subprocess.check_call([sys.executable, "-m", "pip", "install", "shaclgen"])
finally:
    import shaclgen
stream = os.popen('shaclgen output2.owl')
output = stream.read()
print (output)

## shape file generated by SHACLGEN
@prefix ex: <http://www.example.org/> .
@prefix ns1: <http://www.semanticweb.org/svg/ontologies/2021/4/untitled-ontology-3#> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix sh: <http://www.w3.org/ns/shacl#> .

ex:ns1_Game a sh:NodeShape ;
    sh:nodeKind sh:BlankNodeOrIRI ;
    sh:property ex:ns1_appid,
        ex:ns1_hasTag,
        ex:ns1_name,
        ex:ns1_owners,
        ex:ns1_peak_activity,
        ex:ns1_players_amt,
        ex:ns1_playtime,
        ex:ns1_price,
        ex:ns1_rating ;
    sh:targetClass ns1:Game .

ex:ns1_Genre a sh:NodeShape ;
    sh:nodeKind sh:BlankNodeOrIRI ;
    sh:targetClass ns1:Genre .

ex:ns1_Tag a sh:NodeShape ;
    sh:nodeKind sh:BlankNodeOrIRI ;
    sh:targetClass ns1:Tag .

ex:owl_AllDisjointClasses a sh:NodeShape ;
    sh:nodeKind sh:BlankNodeOrIRI ;
    sh:property ex:owl_members ;
 

Валидация:

In [ ]:
try:
    from pyshacl import validate
except:
    subprocess.check_call([sys.executable, "-m", "pip", "install", "pyshacl"])
finally:
    from pyshacl import validate
targ_g = gr
shacl_g = output

validation = validate(targ_g, shacl_g, inference='rdfs', abort_on_error=False)
conforms, report_graph, report_text = validation
report_graph.serialize(destination='after_validation.owl', format='xml')
print(report_text)

Validation Report
Conforms: True



# **Документация**

In [ ]:
try:
    import pylode
except:
    subprocess.check_call([sys.executable, "-m", "pip", "install", "pylode"])
finally:
    import pylode

html = pylode.MakeDocco(input_data_file='output2.owl', outputformat="html").document()
f=open("doc.html", "w")
f.write(html)
f.close()

http://www.semanticweb.org/svg/ontologies/2021/4/untitled-ontology-3#Battle Royale does not look like a valid URI, trying to serialize this will break.
http://www.semanticweb.org/svg/ontologies/2021/4/untitled-ontology-3#Third-Person Shooter does not look like a valid URI, trying to serialize this will break.
http://www.semanticweb.org/svg/ontologies/2021/4/untitled-ontology-3#Online Co-Op does not look like a valid URI, trying to serialize this will break.
http://www.semanticweb.org/svg/ontologies/2021/4/untitled-ontology-3#Early Access does not look like a valid URI, trying to serialize this will break.
http://www.semanticweb.org/svg/ontologies/2021/4/untitled-ontology-3#Third Person does not look like a valid URI, trying to serialize this will break.
http://www.semanticweb.org/svg/ontologies/2021/4/untitled-ontology-3#Massively Multiplayer does not look like a valid URI, trying to serialize this will break.
http://www.semanticweb.org/svg/ontologies/2021/4/untitled-ontology-3#Free to

# **VoID**

In [ ]:
from rdflib.void import generateVoID
prefix=dict(gr.namespaces())['']
gr.add((prefix, rdflib.namespace.DCTERMS.title, Literal('Steam knowledge base')))
gr.add((prefix, rdflib.namespace.DCTERMS.publisher, Literal('Vsevolod Sergeev')))
void_gr, _ = generateVoID(gr)
print(void_gr.serialize().decode('utf-8'))

<?xml version="1.0" encoding="UTF-8"?>
<rdf:RDF
   xmlns:ns1="http://rdfs.org/ns/void#"
   xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#"
>
  <rdf:Description rdf:about="http://example.org/Dataset_property7">
    <rdf:type rdf:resource="http://rdfs.org/ns/void#Dataset"/>
    <ns1:triples rdf:datatype="http://www.w3.org/2001/XMLSchema#integer">8</ns1:triples>
    <ns1:properties rdf:datatype="http://www.w3.org/2001/XMLSchema#integer">1</ns1:properties>
    <ns1:property rdf:resource="http://www.semanticweb.org/svg/ontologies/2021/4/untitled-ontology-3#peak_activity"/>
    <ns1:entities rdf:datatype="http://www.w3.org/2001/XMLSchema#integer">8</ns1:entities>
    <ns1:classes rdf:datatype="http://www.w3.org/2001/XMLSchema#integer">1</ns1:classes>
    <ns1:distinctSubjects rdf:datatype="http://www.w3.org/2001/XMLSchema#integer">8</ns1:distinctSubjects>
    <ns1:distinctObjects rdf:datatype="http://www.w3.org/2001/XMLSchema#integer">3</ns1:distinctObjects>
  </rdf:Description>
  <r